<a href="https://www.kaggle.com/code/amirasadisamani/linear-regression-in-python?scriptVersionId=202405464" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **Linear Regression in Python**

**Code Attribution and Modifications**

This code was originally developed by **Dr. Minooei** as part of the Data Science course at **Tosea Institute**. I have made minor changes to the original code to adapt it for a similar project. My intention in sharing this version is to provide a useful reference for others working on related projects and to contribute to the data science community.

All credit for the foundational structure of the code goes to Dr. Minooei and the Tosea Institute. Any modifications or adaptations made are my own and are intended to enhance the original work for broader use.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import statsmodels.api as sm

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


## Read Data

In [ ]:
car_data = pd.read_csv('/kaggle/input/d/amirasadisamani/car-price-prediction/car_price.csv')

In [ ]:
car_data.head()

**Comprehension of the Business Inquiry**

Suggested Pricing for Pre-owned Vehicles

Marketing Criteria: **80%** of forecasts should range within **-15%** and **+15%** of the real price

## **Data inspection**

In [ ]:
car_data.info()

In [ ]:
car_data.isna().sum()

In [ ]:
car_data.columns

**Price**    : Sales price in Euro

**Age**      : Age of a used car in month

**KM**       : Kilometerage

**FuelType** : Petrol, Diesel, CNG

**HP**       : Horse power

**MetColor** : 1 : if Metallic color, 0 : Not

**Automatic**: 1 : if Aoutomatic, 0 : Not

**CC**       : Engine displacement in cc

**Doors**   : # of doors

**Weight**   : Weight in Kilogram

In [ ]:
# Gain a better understanding of data and learn how to interpret labeled data and features."
car_data.describe()

In [ ]:
car_data.info()

**Continuous variables distribution**

In [ ]:
var_ind = [0, 1, 2, 4, 7, 9]
plot = plt.figure(figsize=(12,6))
plot.subplots_adjust(hspace = 0.5, wspace = 0.5)
for i in range(1, 7):
    a = plot.add_subplot(2, 3, i)
    a.hist(car_data.iloc[: , var_ind[i - 1]], alpha = 0.7)
    a.title.set_text('Histogram of ' + car_data.columns[var_ind[i - 1]])

**Box plot of price**

In [ ]:
plt.boxplot(car_data['Price'], showmeans = True)
plt.title('Boxplot of Price')
plt.show()

**Correlation analysis**

In [ ]:
cont_var = ['Price', 'Age','KM', 'HP','CC', 'Weight']
corr_table = round(car_data[cont_var].corr(method = 'pearson'), 2)
corr_table

**Scatter Plot**

In [ ]:
var_ind = [1, 2, 4, 7, 9]
plot = plt.figure(figsize = (12, 6))
plot.subplots_adjust(hspace = 0.5, wspace = 0.5)
for i in range(1, 6):
    a = plot.add_subplot(2, 3, i)
    a.scatter(x = car_data.iloc[: , var_ind[i - 1]], y = car_data.iloc[: , 0], alpha = 0.5)
    a.title.set_text('Price vs. ' + car_data.columns[var_ind[i - 1]])

**Categorical variables**

In [ ]:
car_data['FuelType'].value_counts()

In [ ]:
car_data["MetColor"].value_counts()

In [ ]:
car_data["Automatic"].value_counts()

In [ ]:
car_data["Doors"].value_counts()

In [ ]:
car_data.loc[car_data['Doors'] == 2,:]

**Note:** *Small sample for 2-door cars and CNG*

## Data Preparation

In [ ]:
car_data.head()

**Divide Dataset into Train and Test**

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(car_data, test_size= 0.3, random_state= 247)

In [ ]:
train.shape

In [ ]:
test.shape

## Build Linear Regression Models



**Model 1: Simple Linear Regression**

In [ ]:
#Price vs KM
#Define the feature set X 

X_train = train['KM']
X_train = sm.add_constant(X_train) # adding a constant

y_train = train['Price']

In [ ]:
X_train.head()

In [ ]:
y_train.head()

I prefer to use the class OLS of "statsmodels" library for linear regression instead of "sikitlearn" because the results from "statmodels" are more detailed to understand and tune for better results.

In [ ]:
#Regression Model
regmodel = sm.OLS( y_train, X_train)
regmodel_1 = regmodel.fit()
regmodel_1.summary()

results :
- R-squared: low R-squared because of using just one feature
- R-squared < 0.05 There is a linear relation between KM and Price
- P>|t| < 0.05 : KM is a significant feature
- Prob(Omnibus) and Jarque-Bera (JB) > 0.05 : Residuals distribution are not normal - bad model

In [ ]:
#Check Assumptions of Regression
#Normality of residuals
regmodel_1.resid

In [ ]:
#Function to plot histogram of residuals
def hist_residuals(model, bins = 50):
    #Calculate density
    from scipy import stats
    density = stats.gaussian_kde(model.resid)
    xp = np.linspace(model.resid.min(), model.resid.max(), 100)
    yp = density.pdf(xp)

    #Histogram
    plt.hist(model.resid, bins = bins, 
             color = 'red', alpha = 0.7, density = True)
    plt.axvline(model.resid.mean(), color = 'black', 
                linewidth = 2, linestyle = '--', label = "Average")
    plt.title('Histogram of Residuals')
    plt.xlabel('Residuals')
    plt.ylabel('Density')
    plt.plot(xp, yp, color = 'black', linewidth = 2)
    plt.legend()
    
    return plt.show()

In [ ]:
hist_residuals(regmodel_1)

In [ ]:
#QQ-plot
sm.qqplot(regmodel_1.resid, line = 's')
plt.show()

In [ ]:
#Test for Skewness and Kurtosis
#Good for sample size > 25

#Jarque-Bera Test (Skewness = 0 ?)
#H0: the data is normally distributed
#p-value < 0.05 reject normality assumption

#Omnibus K-squared normality test
#The Omnibus test combines the random variables for 
# Skewness and Kurtosis into a single test statistic
#H0: the data is normally distributed
#p-value < 0.05 reject normality assumption

regmodel_1.summary()

**Note: Residuals are not Normally Distributed!**

In [ ]:
#Function to plot residuals vs. fitted values
def residuals_fittedvalues_plot(model):
    #Implement Lowess algorithm
    lowess_res = sm.nonparametric.lowess(model.resid, model.fittedvalues)

    #Scatter plot: residuals vs. fitted values
    plt.scatter(x = model.fittedvalues, y = model.resid, 
                color = 'red', alpha = 0.7, label = 'data')
    plt.title('Residuals vs. Fitted values')
    plt.xlabel('Fitted Values', fontsize = 12)
    plt.ylabel('Residuals', fontsize = 12)
    plt.grid()

    #Add LOWESS line
    plt.plot(lowess_res[:, 0], lowess_res[:, 1], 'black', 
             alpha = 0.8, linewidth = 2, label = 'LOWESS')
    plt.legend()

    #Top three observations with greates absolute value of the residual
    top3 = abs(model.resid).sort_values(ascending = False)[:3]
    for i in top3.index:
        plt.annotate(i, xy = (model.fittedvalues[i], model.resid[i]), color = 'blue')
    
    return plt.show()

In [ ]:
#Residuals vs. fitted values
residuals_fittedvalues_plot(regmodel_1)

There is evidence of heteroscedasticity, and the residuals display a pattern, which violates the assumptions of linear regression.

In [ ]:
#Function to check Cook's distance
def influencer_detector(model, thershold = 1):
    
    #create instance of influence
    influence = model.get_influence()

    #Obtain Cook's distance for each observation
    cooks = influence.cooks_distance

    #Check observations w/ Cook's distance greater than thershold
    return np.where(cooks[0] > 1)

In [ ]:
#Check Cook's ditance - model 1
influencer_detector(regmodel_1)

Cook's distance indicates the influence of each observation on the fitted response values. In this case, there is no observation with significant influence.

## Model 2: Quadratic Regression

According to the plot of KM vs. Price, the relation between them is not a smooth linear, and a curved line better describes the relation between them. Therefore, we will try quadratic regression.

In [ ]:
train['KM_2'] = train['KM'] ** 2
train.head()

In [ ]:
train['KM_2'].isna().sum()

In [ ]:
#Define the feature set X 
X_train = train[['KM', 'KM_2']]
X_train = sm.add_constant(X_train) # adding a constant

#Define response variable
y_train = train['Price']

In [ ]:
X_train.head()

In [ ]:
#Regression Model
regmodel = sm.OLS(y_train, X_train)
regmodel_2 = regmodel.fit()
regmodel_2.summary()

In [ ]:
#Check Assumptions of Regression
#Normality of residuals
hist_residuals(regmodel_2)

In [ ]:
#QQ-plot
sm.qqplot(regmodel_2.resid, line = 's')
plt.show()

In [ ]:
#Test for Skewness and Kurtosis
#Good for sample size > 25

#Jarque-Bera Test (Skewness = 0 ?)
#H0: the data is normally distributed
#p-value < 0.05 reject normality assumption

#Omnibus K-squared normality test
#The Omnibus test combines the random variables for 
# Skewness and Kurtosis into a single test statistic
#H0: the data is normally distributed
#p-value < 0.05 reject normality assumption

regmodel_2.summary()

In [ ]:
#Residuals vs. Fitted Values
residuals_fittedvalues_plot(regmodel_2)

In [ ]:
#Check Cook's ditance - model 2
influencer_detector(regmodel_2)

In [ ]:
#Linear vs Quadratic Regression
plt.scatter(x = train['KM'], y = train['Price'], alpha = 0.6)

#Linear Regression
params1 = np.polyfit(train['KM'], train['Price'], 1)
xp = np.linspace(train['KM'].min(), train['KM'].max(), 100)
yp1 = np.polyval(params1, xp)
plt.plot(xp, yp1, alpha = 0.9, linewidth = 2, 
         color = 'green', label = 'Linear Regression')

#Quadratic Regression
params2 = np.polyfit(train['KM'], train['Price'], 2)
yp2 = np.polyval(params2, xp)
plt.plot(xp, yp2, alpha = 0.9, linewidth = 2, 
         color = 'red', label = 'Quadratic Regression')

plt.xlabel('KM', fontsize = 12)
plt.ylabel('Price', fontsize = 12)
plt.title('Price vs. KM', fontsize = 12)
plt.legend()
plt.show()

In [ ]:
#Check Multicollinearity
#Import library for VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):
    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X, i) for i in range(X.shape[1])]
    return(vif)

In [ ]:
calc_vif(X_train)
#If VIF > 10 then multicollinearity is high

To address the issue of multicollinearity in this case, we can scale the features.

In [ ]:
#Scaled Variable
train['KM_Scaled']   = (train['KM'] - train['KM'].mean()) / train['KM'].std()
train['KM_Scaled_2'] = train['KM_Scaled'] ** 2
train.head()

In [ ]:
train[['KM_Scaled', 'KM_Scaled_2']].isna().sum()

In [ ]:
#Define the feature set X 
X_train = train[['KM_Scaled', 'KM_Scaled_2']]
X_train = sm.add_constant(X_train) # adding a constant

#Define response variable
y_train = train['Price']

In [ ]:
#Regression Model
regmodel = sm.OLS(y_train, X_train)
regmodel_2 = regmodel.fit()
regmodel_2.summary()

In [ ]:
#Plot histogram of residuals
#Histogram of residuals - model 2
hist_residuals(regmodel_2)

In [ ]:
calc_vif(X_train)
#If VIF > 10 then multicollinearity is high

## Model 3: Use All Variables

In [ ]:
train.info()

In [ ]:
#Change Dtype of categorical columns to object
train[['FuelType', 
       'MetColor', 
       'Automatic', 
       'Doors']] = train[['FuelType', 
                          'MetColor', 
                          'Automatic', 
                          'Doors']].apply(lambda col: col.astype('category'), 
                                          axis = 0)
train.info()

In [ ]:
#Create dummies for columns with categorical variables
dummies = pd.get_dummies(train[['FuelType', 
                                'MetColor', 
                                'Automatic', 
                                'Doors']], dtype = int)
dummies.head()

In [ ]:
train.drop(columns = ['Price', 
                      'KM', 
                      'KM_2',
                      'FuelType', 
                      'MetColor', 
                      'Automatic', 
                      'Doors'])

For a linear regression model, we should have n-1 dummy variables. Therefore, we will remove one of the less critical features from each category variable.

In [ ]:
#Define the feature set X 
X_ = train.drop(columns = ['Price', 
                           'KM', 
                           'KM_2',
                           'FuelType', 
                           'MetColor', 
                           'Automatic', 
                           'Doors'])
X_train = pd.concat([X_, dummies[['FuelType_Diesel', 
                                  'FuelType_Petrol', 
                                  'MetColor_1', 
                                  'Automatic_1',
                                  'Doors_3', 
                                  'Doors_4', 
                                  'Doors_5']]], axis = 1)
X_train = sm.add_constant(X_train) # adding a constant

#Define response variable
y_train = train['Price']

In [ ]:
X_train.head()

In [ ]:
#Regression Model
regmodel = sm.OLS(y_train, X_train)
regmodel_3 = regmodel.fit()
regmodel_3.summary()

We will check the features with high P>|t| values to determine if they should be dropped from the model.

In [ ]:
#Plot histogram of residuals
hist_residuals(regmodel_3)

In [ ]:
#Removing variables: MetColor
train.groupby('MetColor')['Price'].mean()

In [ ]:
#Removing variables: Doors 
train.groupby('Doors')['Price'].mean()

In [ ]:
#Removing variables: HP
train[['Price', 'HP']].corr()

In [ ]:
#Scatter Plot for Price vs. HP
plt.scatter(x = train['HP'], y = train['Price'])
plt.title('Price vs. HP')
plt.xlabel('HP')
plt.ylabel('Price')

The investigation above shows no significant relation between 'Price' and "HP, MetColor, Doors," so we will drop them.

In [ ]:
#Removing variables: HP, MetColor, and Doors
#Define the feature set X 
X_train = X_train.drop(columns = ['HP', 
                                  'MetColor_1',
                                  'Doors_3', 
                                  'Doors_4', 
                                  'Doors_5'])

#Define response variable
y_train = train['Price']

In [ ]:
X_train.head()

In [ ]:
#Regression Model
regmodel = sm.OLS(y_train, X_train)
regmodel_3 = regmodel.fit()
regmodel_3.summary()

Removing the insignificant features did not decrease the fitting metrics, such as R-squared, compared to the last model. It is good news.

In [ ]:
#Check Assumptions of Regression
#Normality of residuals
#Plot histogram of residuals
hist_residuals(regmodel_3)

In [ ]:
#QQ-plot
sm.qqplot(regmodel_3.resid, line = 's')
plt.show()

## Model 4: Improved Multiple Regression

According to the P>|t| results for FuelType_Diesel and FuelType_Petrol, we will simplify the FuelType feature by converting it into binary features.

In [ ]:
train.loc[train['FuelType'] == 'Petrol', 'IfPetrol'] = 'P'
train.loc[train['FuelType'] != 'Petrol', 'IfPetrol'] = 'NP'
train.head()

In [ ]:
train['IfPetrol'].isna().sum()

In [ ]:
#Create dummies for columns with categorical variables
dummies = pd.get_dummies(train[['IfPetrol']], dtype= int)
dummies.head()

In [ ]:
#Define the feature set X 
X_train = X_train.drop(columns = ['FuelType_Diesel', 
                                  'FuelType_Petrol'])
X_train = pd.concat([X_train, dummies['IfPetrol_NP']], axis = 1)

#Define response variable
y_train = train['Price']

In [ ]:
X_train.head()

In [ ]:
#Regression Model
regmodel = sm.OLS(y_train, X_train)
regmodel_4 = regmodel.fit()
regmodel_4.summary()

In [ ]:
#Check Assumptions of Regression
#Normality of residuals
hist_residuals(regmodel_4)

In [ ]:
#QQ-plot
sm.qqplot(regmodel_4.resid, line = 's')
plt.show()

In [ ]:
#Test for Skewness and Kurtosis
#Good for sample size > 25

#Jarque-Bera Test (Skewness = 0 ?)
#H0: the data is normally distributed
#p-value < 0.05 reject normality assumption

#Omnibus K-squared normality test
#The Omnibus test combines the random variables for 
# Skewness and Kurtosis into a single test statistic
#H0: the data is normally distributed
#p-value < 0.05 reject normality assumption

regmodel_4.summary()

In [ ]:
#Residuals vs. Fitted Values
residuals_fittedvalues_plot(regmodel_4)

In [ ]:
#Remove Cases(be cautious!)
#observations with greates residual
regmodel_4.resid.sort_values(ascending = False)[:5]

In [ ]:
regmodel_4.resid.sort_values(ascending = True)[:5]

In [ ]:
#Remove Cases(be cautious!)
X_train2 = X_train.drop(index = [81,111, 113, 283, 292, 490, 543, 849, 947])
y_train2 = y_train.drop(index = [81,111, 113, 283, 292, 490, 543, 849, 947])

In [ ]:
#Regression Model
regmodel = sm.OLS(y_train2, X_train2)
regmodel_4 = regmodel.fit()
regmodel_4.summary()

In [ ]:
#Check Assumptions of Regression
#Normality of residuals
hist_residuals(regmodel_4)

In [ ]:
#QQ-plot
sm.qqplot(regmodel_4.resid, line = 's')
plt.show()

In [ ]:
#Test for Skewness and Kurtosis
#Good for sample size > 25

#Jarque-Bera Test (Skewness = 0 ?)
#H0: the data is normally distributed
#p-value < 0.05 reject normality assumption

#Omnibus K-squared normality test
#The Omnibus test combines the random variables for 
# Skewness and Kurtosis into a single test statistic
#H0: the data is normally distributed
#p-value < 0.05 reject normality assumption

regmodel_4.summary()

In [ ]:
#Residuals vs. Fitted Values
residuals_fittedvalues_plot(regmodel_4)

In [ ]:
#Check Cook's Distance
influencer_detector(regmodel_4)

We will remove 'Automatic_1' because the P-value exceeds the acceptable threshold. (0,05)

In [ ]:
#Final Regression Model

#Remove Automatic
X_train2 = X_train2.drop(columns = ['Automatic_1'])

#Regression Model
regmodel = sm.OLS(y_train2, X_train2)
regmodel_4 = regmodel.fit()
regmodel_4.summary()

In [ ]:
#Number of removed observation < 1%
(X_train.shape[0] - X_train2.shape[0]) / X_train.shape[0] * 100

## Test the Model

In [ ]:
#Coefficients of the Model
regmodel_4.params

In [ ]:
#Extract features
regmodel_4.model.exog_names

In [ ]:
#Confidence Intervals for Model Parameters
regmodel_4.conf_int(alpha = 0.05)

In [ ]:
#Data preparation
test['KM_Scaled'] = (test['KM'] - test['KM'].mean()) / test['KM'].std()
test['KM_Scaled_2'] = test['KM_Scaled'] ** 2
test.loc[test['FuelType'] == 'Petrol', 'IfPetrol'] = 'P'
test.loc[test['FuelType'] != 'Petrol', 'IfPetrol'] = 'NP'
test.head()

In [ ]:
#Create dummies for columns with categorical variables
dummies = pd.get_dummies(test[['IfPetrol']], dtype= int)
X_test  = pd.concat([test[['Age', 
                           'CC', 
                           'Weight', 
                           'KM_Scaled', 
                           'KM_Scaled_2']], 
                     dummies['IfPetrol_NP']], axis = 1)
X_test = sm.add_constant(X_test) # adding a constant
X_test.head()

In [ ]:
X_train2.head()

In [ ]:
#Prediction
test_pred = regmodel_4.predict(X_test)
test_pred

In [ ]:
#Actual vs. Prediction
plt.scatter(x = test['Price'], y = test_pred)
plt.xlabel('Actual')
plt.ylabel('Prediction')
plt.title('Actual vs. Prediction')

#Add 45 degree line
xp = np.linspace(test['Price'].min(), test['Price'].max(), 100)
plt.plot(xp, xp, alpha = 0.9, linewidth = 2, color = 'red')
plt.show()

In [ ]:
#Absolute error mean, median, sd, max, min
abs_error = abs(test['Price'] - test_pred)
abs_error.describe()

In [ ]:
plt.hist(abs_error)
plt.show()

In [ ]:
#Error percentage median, sd, mean, max, min
e_percent = round(abs(test['Price'] - test_pred) / test['Price'] * 100, 2)
e_percent.describe()

In [ ]:
#Marketing Requirement
sum(e_percent <= 15) / len(e_percent) * 100